# &AElig;co
***Calc module***
<br/><br/>

✈️♻️

This code is part of the Master's Thesis "A Life Cycle Approach for Aircraft Ecodesign", by Giácomo Parolin.

``Please enter the following information:``

In [1]:
import os
os.chdir("C:/Users/giparoli/Documents/Projetos/AEco")

input_path = './Data/Antera_inputs_v3.xlsx'
input_sheet = 'Antera_updated'
output_path = './Outputs/'+ input_sheet + '_outputs'
database_path = './Data/database_Antera.xlsx'

iterations = 100
chunks = 'auto'

In [2]:
# from Tools_Iris import *
# from Model_Iris import *

# from Tools_Antera import *
# from Model_Antera import *

from Tools_Antera_upd import *
from Model_Antera_upd import *

In [3]:
inputs = read_inputs(input_path, input_sheet)
p = ParameterSet(inputs, iterations, chunks)
p = func_unit(p)

In [4]:
p.data["ha_fleet"].mean().compute().data

array(1.78235701e+10)

In [5]:
p.data["ha_life"].mean().compute().data

array(2172546.3326)

In [6]:
UP_dataframe = read_unit_processes(database_path)
UP = unit_process_dataset(UP_dataframe)

In [7]:
inventory = LCI(name=input_sheet, iterations=iterations, UP=UP, parameters=p)
inventory.run()

<xarray.Dataset>
Dimensions:         (Substances: 1835, i: 100)
Coordinates:
    Units           (Substances) object 'kg' 'kg' 'kg' 'kg' ... 'kg' 'm3' 'kg'
  * Substances      (Substances) MultiIndex
  - name            (Substances) object '1,4-Butanediol' ... 'Carbon dioxide,...
  - compartment     (Substances) object 'Water' 'Air' 'Air' ... 'Air' 'Air'
  - subcompartment  (Substances) object 'surface water' ... 'lower stratosphe...
  * i               (i) int32 0 1 2 3 4 5 6 7 8 9 ... 91 92 93 94 95 96 97 98 99
Data variables: (12/16)
    Office          (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Infrastructure  (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Capital         (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Materials       (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Factory         (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Logistics       (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    ...              ...
    Fuel            (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Recycling       (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Incineration    (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Landfill        (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Prototypes      (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Certification   (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
Attributes:
    Name:     Antera_updated

In [9]:
MP_data, EP_data = read_CF(database_path)
CF = CharactFactors(MP_data, EP_data, UP.Substances)

In [10]:
CF.MP.dataset['NLT'].data = np.negative(CF.MP.dataset['NLT'].data)
CF.MP.to_array();

In [11]:
aeco = LCIA.build(inventory, CF)

In [12]:
def cc_flight(lca):
    return lca.get_cat("CC")["Flight"].mean("i").compute().data

In [13]:
cc_flight(aeco)

array(0.26292367)

## Midpoint Results

Raw midpoint results can be accessed by the `LCIA.MP` attribute. Mean midpoint results are calculated by the `LCIA.mean` method, with pathway set to "MP".

In [41]:
display(aeco.MP)

<xarray.Dataset>
Dimensions:         (Categories: 18, i: 50000)
Coordinates:
  * i               (i) int32 0 1 2 3 4 5 ... 49995 49996 49997 49998 49999
  * Categories      (Categories) <U3 'FE' 'MRD' 'POF' 'MET' ... 'FD' 'WD' 'CC'
Data variables: (12/16)
    Office          (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Infrastructure  (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Capital         (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Materials       (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Factory         (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Logistics       (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    ...              ...
    Fuel            (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Recycling       (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Incineration    (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Landfill        (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Prototypes      (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Certification   (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>

In [14]:
aeco.mean("MP", by="sum").compute()

<xarray.DataArray 'stack-9786978069e01f90b7ee8b78d22b2a0b' (Categories: 18)>
array([3.47673293e-02, 1.45367512e+06, 9.92854587e-02, 2.27864769e+02,
       3.69835419e-02, 1.29439522e-01, 5.08547577e-02, 1.79333148e+02,
       2.06480898e+02, 5.07248435e-03, 5.20408290e-04, 1.08528287e+00,
       1.63128766e+01, 3.55717401e+05, 8.96515516e-02, 1.52185185e+00,
       9.65473181e-01, 2.28389690e+03])
Coordinates:
  * Categories  (Categories) <U3 'FE' 'MRD' 'POF' 'MET' ... 'IR' 'FD' 'WD' 'CC'

## Endpoint Results

Raw endpoint results can be accessed by the `LCIA.EP` attribute. Mean endpoint results are calculated by the `LCIA.mean` method, with pathway set to "EP".

In [18]:
display(aeco.EP)

<xarray.Dataset>
Dimensions:         (AOP: 3, i: 50000)
Coordinates:
  * i               (i) int32 0 1 2 3 4 5 ... 49995 49996 49997 49998 49999
  * AOP             (AOP) <U2 'HH' 'ED' 'RA'
Data variables: (12/16)
    Office          (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Infrastructure  (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Capital         (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Materials       (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Factory         (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Logistics       (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    ...              ...
    Fuel            (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Recycling       (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Incineration    (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Landfill        (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Prototypes      (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Certification   (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
Attributes:
    Name:     Antera_spray_mogas

In [35]:
aeco.mean("EP", by="sum").compute()

<xarray.DataArray 'stack-b16e029c735df791a3315e7db32c4566' (AOP: 3)>
array([2.24874163e-01, 6.96987565e-05, 1.28715973e-01])
Coordinates:
  * AOP      (AOP) <U2 'HH' 'ED' 'RA'

## Contribution to Variance (CTV)

Calculate CTV results for all inputs with midpoint and endpoint results. CTV results can be acessed by the `LCIA.CTV` attribute.

In [13]:
aeco.build_CTV(parameterset=p)

<xarray.Dataset>
Dimensions:     (Parameters: 101)
Coordinates:
  * Parameters  (Parameters) <U18 'takt_US' 'takt_BR' ... 'p_recycl_CFRP'
Data variables: (12/21)
    FE          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    MRD         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    POF         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    MET         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    TA          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ULO         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ...          ...
    FD          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    WD          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    CC          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    HH          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ED          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    RA          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>

In [16]:
aeco.CTV["CC"].sortby(aeco.CTV["CC"], ascending=False)#.compute()

<xarray.DataArray 'CC' (Parameters: 101)>
dask.array<getitem, shape=(101,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * Parameters  (Parameters) <U18 'pesticide_use' ... 't_cruise_img'

# Export Results

Export MP, EP, LCI and CTV result to NetCDF file.

In [14]:
aeco.save(output_path, LCI=False)

LCIA saved at ./Outputs/Antera_updated_outputs


In [22]:
aeco.save(('./Local/'+ input_sheet + '_outputs with LCI'), LCI=True)

c:\Users\giparoli\Documents\Projetos\AEco\Scripts\Tools_Antera.py:309: RuntimeWarning: invalid value encountered in true_divide
  return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))


LCIA saved at ./Local/Antera_spray_outputs with LCI


In [22]:
#aeco.to_excel(output_path)